In [2]:
import requests
import time


In [4]:
response = requests.get("https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/records/?limit=100&offset=0")
response

<Response [200]>

In [5]:
import json
data = response.json()
data

{'total_count': 475911,
 'results': [{'num_acc': '201500012905',
   'datetime': '2015-01-10T12:15:00+00:00',
   'nom_com': 'Fontenay-le-comte',
   'an': '2015',
   'mois': '01',
   'jour': '10',
   'hrmn': '13:15',
   'lum': 'Plein jour',
   'agg': 'Hors agglomération',
   'int': '1',
   'atm': 'Pluie légère',
   'col': 'Autre collision',
   'dep': '85',
   'com': '092',
   'insee': '85092',
   'adr': None,
   'lat': '4643650',
   'long': '-083389',
   'code_postal': '85200',
   'num': None,
   'coordonnees': {'lon': -0.806673, 'lat': 46.466325},
   'pr': '98',
   'surf': 'mouillée',
   'v1': None,
   'circ': 'A sens unique',
   'vosp': None,
   'env1': '00',
   'voie': '83',
   'larrout': 0,
   'v2': None,
   'lartpc': 0,
   'nbv': 2,
   'catr': 'Autoroute',
   'pr1': 800,
   'plan': 'Partie rectiligne',
   'prof': 'Plat',
   'infra': None,
   'situ': 'Sur bande d’arrêt d’urgence',
   'an_nais': ['1980'],
   'sexe': ['Masculin'],
   'actp': ['Se déplaçant'],
   'grav': ['Blessé'],
   

In [6]:
import pandas as pd
records = data['results']
df = pd.json_normalize(records)
df.head()

,num_acc,datetime,nom_com,an,mois,jour,hrmn,lum,agg,int,...,dep_name,epci_code,epci_name,reg_code,reg_name,com_arm_name,com_code,coordonnees.lon,coordonnees.lat,coordonnees
0,201500012905,2015-01-10T12:15:00+00:00,Fontenay-le-comte,2015,01,10,13:15,Plein jour,Hors agglomération,1,...,Vendée,248500092,CC du Pays de Fontenay-Le-Comte,52,Pays de la Loire,Fontenay-le-Comte,85092,-0.806673,46.466325,NaN
1,201500013845,2015-01-22T07:15:00+00:00,None,2015,01,22,08:15,Plein jour,En agglomération,1,...,Bouches-du-Rhône,241300391,CU de Marseille Provence Métropole (Mpm),93,Provence-Alpes-Côte d'Azur,None,13055,5.443675,43.282410,NaN
2,201500013857,2015-04-10T21:30:00+00:00,None,2015,04,10,23:30,Nuit avec éclairage public allumé,En agglomération,1,...,Bouches-du-Rhône,241300391,CU de Marseille Provence Métropole (Mpm),93,Provence-Alpes-Côte d'Azur,None,13055,5.374423,43.262794,NaN
3,201500013890,2015-04-19T16:05:00+00:00,None,2015,04,19,18:05,Plein jour,En agglomération,1,...,Bouches-du-Rhône,241300391,CU de Marseille Provence Métropole (Mpm),93,Provence-Alpes-Côte d'Azur,None,13055,5.394886,43.295367,NaN
4,201500017899,2015-05-21T15:00:00+00:00,Hemonstoir,2015,05,21,17:00,Plein jour,Hors agglomération,2,...,Côtes-d'Armor,200042471,CC Cideral,53,Bretagne,Hémonstoir,22075,-2.831244,48.158138,NaN


In [9]:
url = f"https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/records/?limit=100&offset=0"
response = requests.get(url)
data = response.json()
records = data['results']
df = pd.json_normalize(records)

In [ ]:
def request_data(offset, max_retries=5, backoff_factor=2):
    url = f"https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/records/?limit=100&offset={offset}"
    retries = 0

    while retries < max_retries:
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()  

            data = response.json()
            records = data.get('results', [])
            df = pd.json_normalize(records)
            return df

        except (requests.RequestException, ValueError) as e:
            wait_time = backoff_factor ** retries
            print(f"[Erreur] Tentative {retries + 1}/{max_retries} : {e}")
            print(f"Pause de {wait_time} secondes avant nouvel essai...")
            time.sleep(wait_time)
            retries += 1

    print("[Erreur] Trop d'échecs, arrêt de la récupération à l’offset", offset)
    return pd.DataFrame()

def fetch_all_data(offset=0, step=100):
    all_data = pd.DataFrame()
    while True:
        df = request_data(offset)
        if df.empty:
            break
        all_data = pd.concat([all_data, df], ignore_index=True)
        offset += step
    return all_data

In [ ]:
def test_request_data(offset, max_retries=5):
    url = f"https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/records/?limit=100&offset={offset}"
    retries = 0
    while retries < max_retries:
        print(offset)
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()  

            data = response.json()
            records = data.get('results', [])
            df = pd.json_normalize(records)
            return df

        except (requests.RequestException, ValueError) as e:
            wait_time = 20
            print(f"[Erreur] Tentative {retries + 1}/{max_retries} : {e}")
            print(f"Pause de {wait_time} secondes avant nouvel essai...")
            time.sleep(wait_time)
            retries += 1

    print("[Erreur] Trop d'échecs, arrêt de la récupération à l’offset", offset)
    return pd.DataFrame()

def test_fetch_all_data(offset=0, step=100):
    all_data = pd.DataFrame()
    i = 0
    while i < 100:
        df = test_request_data(offset)
        if df.empty:
            break
        all_data = pd.concat([all_data, df], ignore_index=True)
        offset += step
        i += 1
    return all_data

In [ ]:
accidents,  = test_fetch_all_data(offset=1000, step=100)

[Erreur] Tentative 1/5 : 400 Client Error: Bad Request for url: https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/records/?limit=100&offset=10000
Pause de 1 secondes avant nouvel essai...
[Erreur] Tentative 2/5 : 400 Client Error: Bad Request for url: https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/records/?limit=100&offset=10000
Pause de 2 secondes avant nouvel essai...
[Erreur] Tentative 3/5 : 400 Client Error: Bad Request for url: https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/records/?limit=100&offset=10000
Pause de 4 secondes avant nouvel essai...
[Erreur] Tentative 4/5 : 400 Client Error: Bad Request for url: https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/records/?limit=100&offset=10000
Pause de 8 secondes avant nouvel e

In [24]:
accidents

,num_acc,datetime,nom_com,an,mois,jour,hrmn,lum,agg,int,...,dep_name,epci_code,epci_name,reg_code,reg_name,com_arm_name,com_code,coordonnees.lon,coordonnees.lat,coordonnees
0,201200016050,2012-09-22T16:30:00+00:00,None,2012,09,22,18:30,Plein jour,En agglomération,1,...,Bouches-du-Rhône,241300391,CU de Marseille Provence Métropole (Mpm),93,Provence-Alpes-Côte d'Azur,None,13055,5.378490,43.298860,NaN
1,201200016928,2012-08-09T05:40:00+00:00,Tarascon,2012,08,09,07:40,Crépuscule ou aube,Hors agglomération,1,...,Bouches-du-Rhône,241300417,CA d'Arles-Crau-Camargue-Montagnette,93,Provence-Alpes-Côte d'Azur,Tarascon,13108,NaN,NaN,NaN
2,201200029866,2012-04-19T10:30:00+00:00,Cholet,2012,04,19,12:30,Plein jour,En agglomération,1,...,Maine-et-Loire,244900700,CA du Choletais,52,Pays de la Loire,Cholet,49099,-0.897218,47.073242,NaN
3,201200030681,2012-06-07T14:34:00+00:00,Nancy,2012,06,07,16:34,Plein jour,En agglomération,1,...,Meurthe-et-Moselle,245400676,CU du Grand Nancy,41,Lorraine,Nancy,54395,6.151262,48.700318,NaN
4,201200028679,2012-12-22T04:25:00+00:00,Le Puy-en-velay,2012,12,22,05:25,Nuit avec éclairage public allumé,En agglomération,1,...,Haute-Loire,244300018,CA du Puy en Velay,83,Auvergne,Le Puy-en-Velay,43157,3.879497,45.045753,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,201500024305,2015-07-16T09:45:00+00:00,Villeneuve-loubet,2015,07,16,11:45,Plein jour,En agglomération,1,...,Alpes-Maritimes,240600585,CA de Sophia Antipolis,93,Provence-Alpes-Côte d'Azur,Villeneuve-Loubet,06161,7.134326,43.631109,NaN
8996,201500024312,2015-11-28T18:30:00+00:00,Villeneuve-loubet,2015,11,28,19:30,Nuit sans éclairage public,En agglomération,1,...,Alpes-Maritimes,240600585,CA de Sophia Antipolis,93,Provence-Alpes-Côte d'Azur,Villeneuve-Loubet,06161,7.121465,43.657949,NaN
8997,201500024373,2015-10-16T14:41:00+00:00,Beziers,2015,10,16,16:41,Plein jour,En agglomération,1,...,Hérault,243400769,CA de Béziers-Méditerranée,91,Languedoc-Roussillon,Béziers,34032,3.246549,43.347866,NaN
8998,201500012649,2015-09-01T15:00:00+00:00,Mundolsheim,2015,09,01,17:00,Plein jour,En agglomération,1,...,Bas-Rhin,246700488,EuroMétropole de Strasbourg,42,Alsace,Mundolsheim,67309,7.715455,48.641633,NaN


In [ ]:
import pandas as pd
import requests
import io

url_csv = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
def download_data(url):
    print("Lancement du téléchargement avec l'URL corrigée. Veuillez patienter...")
    try:
        response = requests.get(url)
        response.raise_for_status()
        csv_data = io.StringIO(response.text)
        df = pd.read_csv(csv_data, sep=';')
        print("Téléchargement et chargement dans le DataFrame terminés avec succès !")
        print("-" * 60)

        nombre_lignes = df.shape[0]
        nombre_colonnes = df.shape[1]

        print(f"Le DataFrame final contient {nombre_lignes:,} lignes et {nombre_colonnes} colonnes.")
        print("-" * 60)
        return df
    except requests.exceptions.RequestException as e:
        print(f"Une erreur de connexion est survenue : {e}")
    except Exception as e:
        print(f"Une erreur inattendue est survenue : {e}")
